### 서울 Airbnb reviews

In [1]:
import json
import re

In [2]:
review = {}
with open('translated_seoul.json','r',encoding='utf-8') as fp:
    review.update(json.load(fp))

In [3]:
review_en={}

for home_id,rev_lsts in review.items():
    review_en[home_id] = []
    for rev_lst in rev_lsts:
        try:
            if str(rev_lst['language'][-2:]) == 'en':
                review_en[home_id].append([rev_lst['rating'],rev_lst['t_comments']])
        except:
            pass

In [5]:
len(review_en)

14608

### Data Preprocessing Step
##### 1. Tokenization
##### 2. Stop word elimination
##### 3. Stemming : 단어를 기본형으로 바꾸어준다. 복수형은 단수형으로, 과거형은 현재형으로 바꾸는 과정
##### 4. Representation

In [4]:
import nltk
from nltk import sent_tokenize
from nltk.stem import WordNetLemmatizer
import tqdm

### Tokenization

In [5]:
# 리뷰별 문장단위 tokenizing
for home_id,rev_lst in tqdm.tqdm(review_en.items()) :
    for rev in rev_lst:
        rev[1] = sent_tokenize(rev[1])

100%|███████████████████████████████████████████████████████████████████████████| 14608/14608 [00:34<00:00, 419.88it/s]


In [6]:
# 길이가 2보다 작은 문장 없애기
for home_id,rev_lst in tqdm.tqdm(review_en.items()) :
    for rev in rev_lst:
        r = []
        for s in rev[1] : 
            if len(s) > 2 :
                r.append(s)
        rev[1] = r        

100%|█████████████████████████████████████████████████████████████████████████| 14608/14608 [00:00<00:00, 26987.24it/s]


###  Stop words elimination

In [7]:
# 소문자와 대문자가 아닌 것은 공백으로 대체
for home_id,rev_lst in review_en.items(): 
    for rev in rev_lst:
        for sentence in rev[1]:
            sentence = re.sub('[^a-zA-Z\s]', '', sentence)

In [8]:
from nltk.corpus import stopwords
stopWords =list(stopwords.words('english'))
print(stopWords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [9]:
add_stopwords = ['airbnb','im','youre','hongdae', 'seoul', 'daegu', 'jeju' ,'busan', 'korea','gangneung',
                 'itaewon','myeongdong','john','gangnam','also','even','dont','namsan','incheon','good','great','little','well','nice']
# additional_stopwords = []
stopWords = set(stopWords + add_stopwords)

In [10]:
# 문장별 단어단위 tokenizing 후 pos_tag
for home_id,rev_lst in tqdm.tqdm(review_en.items()):
    for rev in rev_lst:
        tmp_lst = []
        for sentence in rev[1]:
            token = nltk.regexp_tokenize(sentence.lower() ,"[\w']+")
            no_stopword_token = [w for w in token if not w in stopWords]
            sent_pos = nltk.pos_tag(no_stopword_token)
            tmp_lst.append(sent_pos)
        rev[1] = tmp_lst

100%|██████████████████████████████████████████████████████████████████████████| 14608/14608 [2:10:56<00:00,  1.86it/s]


### 명사, 형용사, 동사, 부사 lemmatization

In [11]:
lemm = WordNetLemmatizer()

def njvr_lemmantizer(sent):
    global lemm
    
    lemm_sent = []
    for word_pos in sent:
        word, pos = word_pos
        if pos[0] == 'N':
            lemm_sent.append(lemm.lemmatize(word,pos='n').lower() +'_N')
        elif pos[0] == 'J':
            lemm_sent.append(lemm.lemmatize(word,pos='a').lower() +'_J')
        elif pos[0] == 'V':
            lemm_sent.append(lemm.lemmatize(word,pos='v').lower() +'_V')
        elif pos[0] == 'R':
            lemm_sent.append(lemm.lemmatize(word,pos='r').lower() +'_R')
        else:pass
    return lemm_sent

In [12]:
for listing_id, rev_lst in tqdm.tqdm(review_en.items()):
    for rev in rev_lst:
        rev_tmp = []
        for sent in rev[1]:
            rev_tmp.append(njvr_lemmantizer(sent))
        rev[1] = rev_tmp

100%|███████████████████████████████████████████████████████████████████████████| 14608/14608 [00:29<00:00, 502.09it/s]


In [13]:
with open('review_postag_seoul2.json','w',encoding='utf-8') as fp:
        json.dump(review_en,fp)

### LDA 

In [14]:
import pandas as pd
import numpy as np
import time

In [15]:
import gensim
from gensim import corpora, models
from gensim.models import CoherenceModel

C:\Users\asd00\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [16]:
review = review_en

In [17]:
texts = []
for home_id,rev_lst in review.items():
    for rev in rev_lst:
        for sentence in rev[1] :
            texts.append(sentence)

In [18]:
texts[:2]

[['september_V',
  'accompany_V',
  'son_N',
  'south_J',
  'go_V',
  'son_N',
  'watch_V',
  'photo_J',
  'reservation_N',
  'second_J',
  'house_N',
  'leave_V',
  'gate_J',
  'national_J',
  'university_N',
  "people's_N",
  'republic_J',
  'china_V',
  'national_J',
  'university_N',
  "people's_N",
  'republic_J',
  'china_N',
  'midnight_N',
  'midnight_N',
  'midnight_N',
  'arrive_V',
  'midnight_R',
  'think_N',
  'hostess_N',
  'hostess_N',
  'wait_V',
  'hospitable_J',
  'host_N',
  'keep_V',
  'second_J',
  'floor_N',
  'clean_J',
  'quiet_J',
  'neat_N',
  'elegant_J',
  'good_J',
  'bed_N',
  'see_V',
  'internet_J',
  'next_J',
  'day_N',
  'bright_J',
  'noise_N',
  'city_N',
  'quiet_J',
  'pastoral_J',
  'nature_N',
  'countryside_N',
  'beautiful_J',
  'view_N',
  'villa_N',
  'green_J',
  'grass_N',
  'small_J',
  'hill_N',
  'surround_V',
  'hill_N',
  'speak_J',
  'english_J',
  'fluently_R',
  'ardent_J',
  'host_N',
  'speak_V',
  'korean_J',
  'english_J',
  't

In [19]:
dic = corpora.Dictionary(texts)

In [20]:
corpus = [dic.doc2bow(text) for text in texts]

In [21]:
len(corpus)

1124076

In [22]:
# corpus: the corpus used to train the topic model.
# id2word: a dictionary that maps word numerical identifiers to word strings 
# alpha: the topic distribution. If we set it to 'auto', then we tell the algorithm to determine the topic proportions automatically.
# num_topics: how many topics the algorithm should find.
# passes: how many times the algorithm should go through the corpus. If you are using a large corpus , then this can be set to 1
# model.num_topics: the number of topics.
# model.show_topic(topic_number, topn) returns the probability distribution over words in one topic. for instance [(0.06, 'food'), (0.04, 'cook'), (0.03, 'fry'), ...]
# model.alpha[topic_number]: the prominence of a topic.

#%%time
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 6, id2word = dic, passes=30 ,iterations = 3000,alpha='symmetric', eta='auto') #LDA model 생성

C:\Users\asd00\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


In [23]:
ldamodel.save('lda_seoul.lda')

#### Airnb category : 정확성 / 의사소통 / 청결도 / 위치 / 체크인 / 가치

In [24]:
ldamodel.show_topics(num_words=10) #토픽을 구성하는 주요단어들

[(0,
  '0.061*"clean_J" + 0.052*"room_N" + 0.048*"location_N" + 0.041*"comfortable_J" + 0.037*"place_N" + 0.036*"apartment_N" + 0.028*"house_N" + 0.023*"clean_N" + 0.018*"quiet_J" + 0.015*"really_R"'),
 (1,
  '0.039*"everything_N" + 0.030*"room_N" + 0.023*"provide_V" + 0.023*"need_V" + 0.016*"thanks_N" + 0.016*"big_J" + 0.016*"bed_N" + 0.015*"question_N" + 0.014*"give_V" + 0.014*"always_R"'),
 (2,
  '0.039*"recommend_V" + 0.032*"place_N" + 0.026*"home_N" + 0.022*"highly_R" + 0.018*"house_N" + 0.017*"stay_V" + 0.015*"experience_N" + 0.014*"korean_J" + 0.013*"make_V" + 0.013*"family_N"'),
 (3,
  '0.056*"time_N" + 0.034*"stay_V" + 0.033*"go_V" + 0.029*"come_V" + 0.026*"next_J" + 0.023*"back_R" + 0.021*"day_N" + 0.021*"thank_N" + 0.020*"night_N" + 0.019*"definitely_R"'),
 (4,
  '0.067*"station_N" + 0.027*"location_N" + 0.025*"restaurant_N" + 0.025*"easy_J" + 0.025*"convenient_N" + 0.023*"walk_V" + 0.023*"subway_N" + 0.021*"minute_N" + 0.017*"area_N" + 0.017*"store_N"'),
 (5,
  '0.084*"host

In [3]:
ldamodel = gensim.models.ldamodel.LdaModel.load('lda_seoul.lda')

In [25]:
import pyLDAvis
import pyLDAvis.gensim 
import matplotlib.pyplot as plt

In [26]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dic)

C:\Users\asd00\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [28]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.239526  0.167907       1        1  20.057068
1     -0.107818 -0.404840       2        1  16.870823
0     -0.226058  0.104909       3        1  16.797932
3      0.221325 -0.053895       4        1  16.066092
5      0.156652  0.106271       5        1  15.768917
2      0.195424  0.079648       6        1  14.439165, topic_info=     Category           Freq           Term          Total  loglift  logprob
term                                                                        
32    Default   86847.000000         host_N   86847.000000  30.0000  30.0000
147   Default   87808.000000      station_N   87808.000000  29.0000  29.0000
184   Default   73759.000000         stay_N   73759.000000  28.0000  28.0000
11    Default   67306.000000        clean_J   67306.000000  27.0000  27.0000
83    Default   58917.000000         time_N   58917.000000  26.0000  26.0000
115   Default   91023.000000         room_N   91023.000000  25.0000  25.0000
292   Default   87953.000000     location_N   87953.000000  24.0000  24.0000
221   Default   44812.000000  comfortable_J   44812.000000  23.0000  23.0000
112   Default   42666.000000   everything_N   42666.000000  22.0000  22.0000
439   Default   37057.000000    recommend_V   37057.000000  21.0000  21.0000
25    Default   43392.000000           go_V   43392.000000  20.0000  20.0000
123   Default   66221.000000         stay_V   66221.000000  19.0000  19.0000
165   Default   30176.000000         come_V   30176.000000  18.0000  18.0000
235   Default   68351.000000    apartment_N   68351.000000  17.0000  17.0000
223   Default   29274.000000         kind_N   29274.000000  16.0000  16.0000
108   Default   33386.000000   restaurant_N   33386.000000  15.0000  15.0000
328   Default   32576.000000         easy_J   32576.000000  14.0000  14.0000
110   Default   30766.000000         walk_V   30766.000000  13.0000  13.0000
47    Default   26994.000000         next_J   26994.000000  12.0000  12.0000
125   Default   24854.000000         home_N   24854.000000  11.0000  11.0000
330   Default   29761.000000       subway_N   29761.000000  10.0000  10.0000
218   Default   41476.000000   convenient_N   41476.000000   9.0000   9.0000
234   Default   25258.000000      provide_V   25258.000000   8.0000   8.0000
93    Default  151817.000000        place_N  151817.000000   7.0000   7.0000
205   Default   25188.000000         need_V   25188.000000   6.0000   6.0000
72    Default   24393.000000         back_R   24393.000000   5.0000   5.0000
466   Default   24712.000000        clean_N   24712.000000   4.0000   4.0000
304   Default   27312.000000       minute_N   27312.000000   3.0000   3.0000
87    Default   68365.000000       really_R   68365.000000   2.0000   2.0000
81    Default   22180.000000        thank_N   22180.000000   1.0000   1.0000
...       ...            ...            ...            ...      ...      ...
619    Topic6    5279.402344         rest_N    5279.526855   1.9352  -5.1860
129    Topic6    5216.342285         care_N    5216.466797   1.9352  -5.1980
1717   Topic6    5049.062988        staff_N    5049.186035   1.9352  -5.2306
77     Topic6    4937.409180  hospitality_N    4937.533203   1.9352  -5.2529
97     Topic6    4871.275879  traditional_J    4871.398926   1.9352  -5.2664
159    Topic6    4634.236328    delicious_J    4634.360352   1.9352  -5.3163
563    Topic6    4525.694824        value_N    4525.818359   1.9352  -5.3400
894    Topic6    4155.301270         size_N    4155.423828   1.9352  -5.4254
577    Topic6    4014.599365        money_N    4014.722168   1.9352  -5.4598
357    Topic6    3961.799316     everyone_N    3961.922363   1.9352  -5.4731
510    Topic6    3731.553467         cold_J    3731.676270   1.9352  -5.5329
854    Topic6    3710.462158          fun_N    3710.585205   1.9352  -5.5386
445    Topic6    3566.802490        sleep_V    3566.925781  

In [27]:
pyLDAvis.save_html(vis,'LDA_seoul.html')